In [5]:
import numpy as np

In [7]:
# replace with any text file containing full set of data
mozart_data = 'txt-files/notewise/custom/mozart.txt'

with open(mozart_data, 'r') as file:
    text = file.read()

In [19]:
# get vocabulary set
vocab = sorted(tuple(set(text.split())))
n = len(words)

# create word-integer encoder/decoder
word2int = dict(zip(vocab, list(range(n))))
int2word = dict(zip(list(range(n)), vocab))

# encode all words in dataset into integers
encoded = np.array([word2int[word] for word in text.split()])

# get vocab_size
vocab_size=len(vocab)

In [ ]:
# split dataset into 90% train and 10% using index
val_idx = int(len(encoded) * (1 - 0.1))
train_data, val_data = encoded[:val_idx], encoded[val_idx:]

In [23]:
len(train_data)/128

79853.9765625

In [9]:
def get_batches(arr, n_seqs, n_words):
    """
        create generator object that returns batches of input (x) and target (y).
        x of each batch has shape 128*128*149 (batch_size*seq_len*vocab_size).
        
        accepts 3 arguments:
        1. arr: array of words from text data
        2. n_seq: number of sequence in each batch (aka batch_size)
        3. n_word: number of words in each sequence
    """
    
    # compute total elements / dimension of each batch
    batch_total = n_seqs * n_words
    
    # compute total number of complete batches
    n_batches = arr.size//batch_total
    
    # chop array at the last full batch
    arr = arr[: n_batches* batch_total]
    
    # reshape array to matrix with rows = no. of seq in one batch
    arr = arr.reshape((n_seqs, -1))
    
    # for each n_words in every row of the dataset
    for n in range(0, arr.shape[1], n_words):
        
        # chop it vertically, to get the input sequences
        x = arr[:, n:n+n_words]
        
        # init y - target with shape same as x
        y = np.zeros_like(x)
        
        # targets obtained by shifting by one
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], x[:, n+n_words]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        
        # yield function is like return, but creates a generator object
        yield x, y   

In [12]:
for i, (x, y) in enumerate(get_batches(train_data, 128, 128)):
    print(x)

[[ 68 102 108 ..., 135  19 124]
 [124  82 135 ..., 135  17  25]
 [ 72  90  95 ...,  21  33 124]
 ..., 
 [ 35  44 124 ..., 135  17  30]
 [ 94 142  12 ..., 142  14  32]
 [ 22 124  83 ...,  90 135  28]]
[[ 68 135   6 ...,  25  35 124]
 [ 47 124  73 ..., 124 103 135]
 [ 78  82  91 ...,  81 135  19]
 ..., 
 [124  80  91 ...,  42 124 118]
 [ 37 124 123 ...,  97 142  35]
 [124  89 135 ..., 124  64 135]]
[[ 81  86  94 ..., 108 135  16]
 [ 41 124 101 ...,  30  46 124]
 [124  86 135 ...,  18 124  89]
 ..., 
 [ 77  86  93 ..., 124  65  78]
 [124 100 142 ..., 100 142   6]
 [  2 124 123 ..., 124  79 135]]
[[ 46 124  76 ..., 135  53 124]
 [ 95 105 135 ...,  41 124  71]
 [135  27 124 ...,  11  33 124]
 ..., 
 [ 82  88  95 ..., 124 101 142]
 [ 19 124  72 ...,  67  80 142]
 [ 17 124  77 ...,  45 124  74]]
[[108 135  46 ...,  35 124  94]
 [106 135   9 ..., 135  20 124]
 [ 83  97 135 ..., 105 135  24]
 ..., 
 [ 39 124  99 ..., 135  52 124]
 [  5  18 124 ..., 144  24  32]
 [135  12 124 ..., 124  98 135]]


In [13]:
x.shape

(128, 128)